In [0]:
from pyspark.sql import SQLContext, SparkSession, Window, Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

r_fraud_annotations             = spark.table('annotations.cred_annotations')
tab_desenrola = (r_fraud_annotations
                 .select('*')
                 .filter(col('annotation_description').isin('flag_desenrola_materialidade'))
                 .withColumnRenamed('document', 'cpf')
                 .limit(10)
                 .select('*', lit(1).alias('is_user_desenrola'))
)
tab_desenrola.toPandas()

In [0]:
from datetime import datetime, timedelta

dataCriacaoConta = "2024-02-26T10:57:00.888Z"
dataCriacaoContaDate = datetime.strptime(dataCriacaoConta,"%Y-%m-%dT%H:%M:%S.%fZ")

d = dataCriacaoContaDate - timedelta(days=61)

print(d)

In [0]:
import os

# Me mostre o caminho absoluto para a pasta arquivos dessa pasta
folder_path = os.path.abspath('arquivos')
folder_path = str(folder_path)

print(folder_path)

In [0]:
import json
pasta = "/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/"
file_name = "arquivos/payload_inicial.txt"
file_path = pasta + file_name
with open(file_path,'r') as f:
    x = f.read()
    jsonzada = json.loads(x)


In [0]:
jsonzada

In [0]:
jsonzada['idSolicitacao']

In [0]:
def execucaoModular (payload_entrada, modulo):
    
    lista_payloads = payload_entrada

    lista_resultado = []

    for caso in lista_payloads:
        payload = json.load(caso) 
        payload = modulo(payload)
        lista_resultado.append(payload)

    return lista_resultado

In [0]:
from datetime import datetime
from datetime import date
import pandas as pd

hojeDate = pd.Timestamp(date.today()) # Avaliar como pegar o dia da execução, pois teremos problemas no Pós acomp pois talvez a data da execução não será a mesma na qualavaliaremos a log
dataCriacaoConta = "2024-02-26T17:18:43.888Z"
######################################################################################################################
if dataCriacaoConta == "":
    flagClienteExistente = False
else:
    dataCriacaoContaDate = datetime.strptime(dataCriacaoConta,"%Y-%m-%dT%H:%M:%S.%fZ")
    delta = hojeDate - dataCriacaoContaDate
    print(delta)
    if delta.days > 60:
        flagClienteExistente = True
    else:
        flagClienteExistente = False
print(flagClienteExistente)

In [0]:
dbutils.library.restartPython()

In [0]:
import sherlock

In [0]:
import pandas as pd
# Import the csv file into a pandas dataframe
pasta  = "/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/arquivos/tabelasHomologacao/"
#
arquivo = "Ja_Cliente_Segmentacao_codigo_motor.csv"
file_path = pasta + arquivo
pd.set_option('display.max_rows', 5)
tabela = pd.read_csv(file_path)
tabela

In [0]:
rangeScoreInterno = "R1"
segmentacaoPolitica = "P1 - Funcionarios"
faixaRendaBruta = "a. < 4k"
flagFuncionario = 1
cpf5e6Digito = 99

# Faixas de CPF
if (0 <= cpf5e6Digito <= 44):
    faixaCPF = "0 <= ..<= 44"
if (45 <= cpf5e6Digito <= 89):
    faixaCPF = "45 <= ..<= 89"
if (90 <= cpf5e6Digito <= 99):
    faixaCPF = "90 <= ..<= 99"

# Primeiro Filtro - Nível rangeScoreInterno
filtroNivelRangeScoreInterno = tabela.loc[(tabela['rangeScoreInterno'] == rangeScoreInterno)]
if filtroNivelRangeScoreInterno.empty:
    filtroNivelRangeScoreInterno = tabela.loc[(tabela['rangeScoreInterno'] == "QUALQUER")]

# Segundo Filtro - Nível segmentacaoPolitica
filtroNivelSegmentacaoPolitica = filtroNivelRangeScoreInterno.loc[(filtroNivelRangeScoreInterno['segmentacaoPolitica'] == segmentacaoPolitica)]
if filtroNivelSegmentacaoPolitica.empty:
    filtroNivelSegmentacaoPolitica = filtroNivelRangeScoreInterno.loc[(filtroNivelRangeScoreInterno['segmentacaoPolitica'] == "QUALQUER")]

# Terceiro Filtro - Nível faixaRendaBruta
filtroNivelFaixaRendaBruta = filtroNivelSegmentacaoPolitica.loc[(filtroNivelSegmentacaoPolitica['faixaRendaBruta'] == faixaRendaBruta)]
if filtroNivelFaixaRendaBruta.empty:
    filtroNivelFaixaRendaBruta = filtroNivelSegmentacaoPolitica.loc[(filtroNivelSegmentacaoPolitica['faixaRendaBruta'] == "QUALQUER")]

# Quarto Filtro - Nível flagFuncionario
filtroNivelflagFuncionario = filtroNivelFaixaRendaBruta.loc[(filtroNivelFaixaRendaBruta['flagFuncionario'] == flagFuncionario)]
if filtroNivelflagFuncionario.empty:
    filtroNivelflagFuncionario = filtroNivelFaixaRendaBruta.loc[(filtroNivelFaixaRendaBruta['flagFuncionario'] == "QUALQUER")]

# Quinto Filtro - Nível 5e6 Digito CPF
filtroNivelCPF = filtroNivelflagFuncionario.loc[(filtroNivelflagFuncionario['cpf5e6Digito'] == faixaCPF)]
if filtroNivelCPF.empty:
    filtroNivelCPF = filtroNivelflagFuncionario.loc[(filtroNivelflagFuncionario['cpf5e6Digito'] == "QUALQUER")]

linha_tab = filtroNivelCPF
# Teste para ver se após filtrar na tabela o cliente foi encontrado
if linha_tab.empty:
    segmentacao = "-1"
    alavancagem = -1
    limiteTeto = -1
    limitePiso = -1
else:
    segmentacao = linha_tab.iloc[0]['SEGMENTAÇÃO FINAL']
    alavancagem = linha_tab.iloc[0]['alavancagem']
    limiteTeto = linha_tab.iloc[0]['limiteTeto']
    limitePiso = linha_tab.iloc[0]['limitePiso']   

#print(segmentacao, alavancagem, limiteTeto, limitePiso)

In [0]:
filtroNivelflagFuncionario

In [0]:
filtroNivelCPF

In [0]:
import pandas as pd
from datetime import datetime
from datetime import date
dataAdmissao = "2002-11-05T17:18:43.888Z"
hojeDate = pd.Timestamp(date.today())  # Avaliar como pegar o dia da execução, pois teremos problemas no Pós acomp pois talvez a data da execução nãoserá a mesma na qual avaliaremos a log
######################################################################################################################

admissaoDate = datetime.strptime(dataAdmissao, "%Y-%m-%dT%H:%M:%S.%fZ")
print("print 1: ",admissaoDate)
admissaoMeses = (hojeDate.year - admissaoDate.year) * 12 + (hojeDate.month - admissaoDate.month)
print("print 2: ",(hojeDate.year - admissaoDate.year) * 12)
print("print 3: ",(hojeDate.month - admissaoDate.month))



print(admissaoMeses)

In [0]:
dataAdmissao = "2002-11-05T17:18:43.888Z"
hojeDate = pd.Timestamp(date.today())
admissaoDate = datetime.strptime(dataAdmissao, "%Y-%m-%dT%H:%M:%S.%fZ")
admissaoMeses = (hojeDate.year - admissaoDate.year) * 12 + (hojeDate.month - admissaoDate.month)
admissaoMeses

In [0]:
# GERANDO DATA DIFS PARA ADICIONAR NA MASSA

from datetime import datetime, timedelta

today = datetime.today()
target_date = today - timedelta(days=60)
dif60 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

target_date = today - timedelta(days=59)
dif59 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

target_date = today - timedelta(days=61)
dif61 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

print("data de 59 dias :",dif59)
print("data de 60 dias :",dif60)
print("data de 61 dias :",dif61)

In [0]:
# como splitar uma massa em partes iguais

#import numpy as np
#dataframeGrande = {}
#num_dataframes = 3
#
#split_dataframes = np.array_split(dataframeGrande, num_dataframes)
#
#for i, df in enumerate(split_dataframes):
#    df.to_csv(f'dataframe{i}.csv', index=False)

In [0]:
def round_with_steps(number):
    return round(number/50) * 50

round_with_steps(47.33)

In [0]:
today = date.today()

dataCriacaoConta = '2024-01-17T18:35:34.748956Z'
dataCriacaoConta = datetime.datetime.strptime(dataCriacaoConta[:10], "%Y-%m-%d").date()

type(today)

In [0]:
delta = today - dataCriacaoConta
delta.days

In [0]:
import pandas as pd

data = spark.read.parquet("s3://picpay-credit-laboratory/modeling/income/3v5/income3v5_backtest")
distinct_df = data.select("source_income").distinct().toPandas()
distinct_df